In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 
import time
import datetime
import sys
from pprint import pprint
# .envファイルから環境変数を読み込む
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
url = os.getenv('KOUYAKU_SCRAPIN_URL_1') #環境変数からURLを取得
response = requests.get(url) #データを取得
soup = BeautifulSoup(response.text, 'html.parser')

In [15]:
record_df

,title,url
0,にほんブログ村,https://slot.blogmura.com/ranking/in?p_cid=111...


In [20]:
main_content = soup.find(class_="content")
#aタグの中のhref属性を取得
#kiji_url_list = [kiji.find("a").get("href") for kiji in kiji_list]
main_content_df = pd.DataFrame(index=[], columns=['title', 'url', ])
for p_tag in main_content.find_all("a"):
    title = p_tag.text.replace('【静岡県】', '')
    url = p_tag.get("href")
    record_df = pd.DataFrame([[title, url]], columns=['title', 'url' ])
    main_content_df = pd.concat([main_content_df, record_df], axis=0)
main_content_df = main_content_df[main_content_df['title'].str.contains('｜')]
main_content_df

,title,url
0,・イベント公約内容｜アツ姫,https://kumaslot.com/2023/11/18/5978/
0,イベント公約内容｜DMM,https://kumaslot.com/2023/11/18/5980/
0,イベント公約内容｜必勝本,https://kumaslot.com/2023/11/18/5982/
0,・イベント公約内容｜スロパチステーション,https://kumaslot.com/2023/11/18/5985/
0,・イベント公約内容｜ドリスロ・ドリパチ,https://kumaslot.com/2023/11/18/5987/
0,・イベント公約内容｜でちゃう,https://kumaslot.com/2023/11/18/5990/
0,・イベント公約内容｜パチエース,https://kumaslot.com/2023/11/18/5989/
0,・イベント公約内容｜ギャラスタ,https://kumaslot.com/2023/11/18/5993/
0,・イベント公約内容｜イナズマ店長,https://kumaslot.com/2023/11/18/5995/
0,・イベント公約内容｜パチンコ店長のホール攻略,https://kumaslot.com/2023/11/18/5997/


In [36]:
soup.find('h3')

<h3 class="wp-block-heading" id="outline_1__1_1">Ace OVER</h3>

In [ ]:
print(block_soup)

In [76]:
kouyaku_list[0].next_sibling.next_sibling

<p>　ノーマルタイプが分岐営業</p>

In [89]:
scraping_df = pd.DataFrame(index=[], columns=['media_name', 'syuzai_name', 'pledge_text', ])
for h2_text in str(soup).split('<h2')[1:]:
    h2_text = '<h2' + h2_text
    #print(h2_text)
    block_soup = BeautifulSoup(h2_text, 'html.parser')
    media_name = block_soup.find('h2').text
    #print(media_name)
    h3_tag_list = block_soup.find_all('h3')
    for h3_tag in h3_tag_list:
        #print(h3_tag.text)
        try:
            syuzai_name = h3_tag.text
        except:
            syuzai_name = 'エラー'
        try:
            pledge_text = h3_tag.next_sibling.next_sibling.text.replace(' ', '').replace('　', '')
            #print(pledge_text)
        except:
            pledge_text = 'エラー'
        try:
            record_df = pd.DataFrame([[media_name, syuzai_name, pledge_text]], columns=['media_name', 'syuzai_name', 'pledge_text' ])
            scraping_df = pd.concat([scraping_df, record_df], axis=0)
        except:
            pass

    #print(block_soup)
    #break
scraping_df['media_name'].unique()

array(['アツ姫', 'DMM', '必勝本', 'スロパチステーション', 'でちゃう', 'パチエース', 'ギャラスタ',
       'イナズマ店長', 'パチンコ店長のホール攻略', 'りゅーこ', 'あしゅら', 'アツモリ', 'サミー系', 'ジャンバリ',
       'じゃんじゃん', 'WIN6GAME', '公約ありの演者来店', '関連する記事', '管理人プロフィール', 'ブログ内検索'],
      dtype=object)

In [90]:
exclude_list = ['関連する記事', '管理人プロフィール', 'ブログ内検索']
exclude_list

scraping_df = scraping_df[~scraping_df['media_name'].isin(exclude_list)]
scraping_df.to_csv('csv/pre_insert_scraping_df.csv', index=False)

In [ ]:
kouyaku_df_type_1 = pd.DataFrame(index=[],columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])
error_media_name_and_syuzai_name_dict = {}
for kiji_url in  kouyaku_block:
    #time.sleep(1)
    response = requests.get(kiji_url) #データを取得
    groupby_media_soup = BeautifulSoup(response.text, 'html.parser')
    #h2タグからメディア名を取得
    for h2_tag in groupby_media_soup.find_all("h2"):
        if 'イベント公約まとめ' in h2_tag.text:
            print(h2_tag.text)
            media_name = h2_tag.text.replace('のイベント公約まとめ', '')
    #クラス名のないdivタグを取得
    for div_elem in groupby_media_soup.find_all('div'):
        #print(div_elem.get('class'))
        if div_elem.get('class') == None and div_elem.find('h3') != None:
            #print(div_elem.text)
            #h3要素を取得
            #print(div_elem.h3.next_sibling)
            syuzai_name = div_elem.h3.text.replace('の公約', '')
            try:
                image_url = div_elem.h3.next_sibling.img.get('src')
                kouyaku_text = div_elem.h3.next_sibling.next_sibling.text.replace('公約　', '')
                kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
            except:
                try:
                    image_url = ''
                    kouyaku_text = div_elem.h3.next_sibling.text.replace('公約　', '')
                    kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
                except Exception as e:
                    error_media_name_and_syuzai_name_dict[media_name + syuzai_name] = e
                    continue
            #print(syuzai_name,kouyaku_text)
            

    #break

In [ ]:
kouyaku_df_type_1 = pd.DataFrame(index=[],columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])
error_media_name_and_syuzai_name_dict = {}
for kiji_url in  kouyaku_block:
    #time.sleep(1)
    response = requests.get(kiji_url) #データを取得
    groupby_media_soup = BeautifulSoup(response.text, 'html.parser')
    #h2タグからメディア名を取得
    for h2_tag in groupby_media_soup.find_all("h2"):
        if 'イベント公約まとめ' in h2_tag.text:
            print(h2_tag.text)
            media_name = h2_tag.text.replace('のイベント公約まとめ', '')
    #クラス名のないdivタグを取得
    for div_elem in groupby_media_soup.find_all('div'):
        #print(div_elem.get('class'))
        if div_elem.get('class') == None and div_elem.find('h3') != None:
            #print(div_elem.text)
            #h3要素を取得
            #print(div_elem.h3.next_sibling)
            syuzai_name = div_elem.h3.text.replace('の公約', '')
            try:
                image_url = div_elem.h3.next_sibling.img.get('src')
                kouyaku_text = div_elem.h3.next_sibling.next_sibling.text.replace('公約　', '')
                kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
            except:
                try:
                    image_url = ''
                    kouyaku_text = div_elem.h3.next_sibling.text.replace('公約　', '')
                    kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
                except Exception as e:
                    error_media_name_and_syuzai_name_dict[media_name + syuzai_name] = e
                    continue
            #print(syuzai_name,kouyaku_text)
            

    #break

In [95]:
read_csv_df = pd.read_csv('csv\pre_insert_scraping_df.csv')
read_csv_df = read_csv_df[['syuzai_name', 'media_name', 'pledge_text']]
#作成時間のカラムを追加
read_csv_df['created_at'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
read_csv_df['updated_at'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#RANKカラムを追加
read_csv_df['rank'] = None
#no_pledge_visit_countカラムを追加
read_csv_df['no_pledge_visit_count'] = 0
read_csv_df.to_csv('csv\dbeaver_csv_insert_scraping_df.csv', index=False)